In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
    'save_best': False,
    'device': "cuda:0"
}

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "model_train", ENTITY_NAME: "classifier_inference_files"}, # stored detector files from train step
      { STEP_NAME: "data_load", ENTITY_NAME: "meta_cifar10_datasets"} # meta information of dataset from data_load step
    ],
    
    tmp_entities =
    [
        { ENTITY_NAME: "classifier_inference_files" }, # temporary detector files from train step
        { ENTITY_NAME: "classifier_onnx_files"}, # temporary detector onnx files after converting 
        { ENTITY_NAME: "meta_cifar10_datasets"} # extracted temporary meta information of dataset from data_load step
    ],
    
    outputs =
    [
        { ENTITY_NAME: "bento_service" } # stored BentoService
    ],
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# specify all notebook wide libraries imports here
# Sinara lib imports is left in the place of their usage
import json
import os
import os.path as osp
import glob
import torch
import numpy as np
import cv2
import onnxruntime, pickle, shutil

In [ ]:
# run spark
from sinara.spark import SinaraSpark
from sinara.archive import SinaraArchive

spark = SinaraSpark.run_session(0)
archive = SinaraArchive(spark)
SinaraSpark.ui_url()

### Loading classifier inference files from the model_train step 
(weights, configs, test image)

In [ ]:
model_train_inputs = substep.inputs(step_name = "model_train")
data_load_inputs = substep.inputs(step_name = "data_load")
tmp_entities = substep.tmp_entities()
# copy config from previos step to outputs

archive.unpack_files_from_store_to_tmp(store_path=model_train_inputs.classifier_inference_files, tmp_dir=tmp_entities.classifier_inference_files)
archive.unpack_files_from_store_to_tmp(store_path=data_load_inputs.meta_cifar10_datasets, tmp_dir=tmp_entities.meta_cifar10_datasets)

### Select obj_detector weights for converting

In [ ]:
# Selecting a weights file to convert to onnx format (best, latest epoch, etc.)
best_weights_pths = glob.glob(f"{tmp_entities.classifier_inference_files}/*best*")
latest_weights_pths = glob.glob(f"{tmp_entities.classifier_inference_files}/*latest*")

weights_pths = best_weights_pths if substep_params['save_best'] and len(best_weights_pths) > 0 else latest_weights_pths

weights_pths.sort(key=lambda file: osp.getmtime(file))

selected_weights_pth = weights_pths[-1]

## Export to ONNX

### Defining basic variables for export to ONNX

In [ ]:
ONNX_convert_params = step_params["ONNX_convert_params"]
DEVICE              = ONNX_convert_params["device"]
OPSET_VERSION       = ONNX_convert_params["opset_version"]
VERBOSE             = ONNX_convert_params["verbose"]

INPUT_NAMES  = ['input']
OUTPUT_NAMES = ['output']
DYNAMIC_AXES = {'input':
                {0: 'batch',
                 2: 'height',
                 3: 'width'},
                'output': 
                {0: 'batch'}
               }

KEEP_INITIALIZERS_AS_INPUTS = False
EXPORT_PARAMS               = True


test_image_path = osp.join(tmp_entities.classifier_inference_files, "test.png")
test_image = cv2.imread(test_image_path)
INPUT_SIZE = test_image.shape[:2]

with open(osp.join(tmp_entities.meta_cifar10_datasets, 'meta_cifar10_datasets.json'), 'r') as f:
   CATEGORIES = json.load(f)

#### Loading torch model

In [ ]:
net_classifier = torch.load(selected_weights_pth, map_location=torch.device(DEVICE))

#### Converting to onnx file

In [ ]:
dummy_input = np.zeros([1, 3, INPUT_SIZE[0], INPUT_SIZE[1]], dtype=np.float32)  # image zeros by shape [height, width, chanels]
dummy_input = torch.Tensor(dummy_input).to(DEVICE)

out_onnx_filename = osp.basename(selected_weights_pth).replace(".pth", ".onnx")
out_onnx_filepath = osp.join(tmp_entities.classifier_onnx_files, out_onnx_filename)

torch.onnx.export(net_classifier,
                  dummy_input,
                  out_onnx_filepath,
                  verbose=VERBOSE,
                  input_names=INPUT_NAMES,
                  output_names=OUTPUT_NAMES,
                  dynamic_axes=DYNAMIC_AXES,
                  export_params=EXPORT_PARAMS,
                  keep_initializers_as_inputs=KEEP_INITIALIZERS_AS_INPUTS
                 )        

### Pack to REST BentoService

In [ ]:
from bento_service import ModelService
from pre_post_processing import PrePostProcessing

# CATEGORIES = [{"id": class_id+1, "name": class_name} for class_id, class_name in enumerate(CLASSES)]

outputs = substep.outputs()

# copy test image 
test_image_path = osp.join(tmp_entities.classifier_inference_files, "test.png")
onnx_test_image_path =  osp.join(tmp_entities.classifier_onnx_files, "test.png")
shutil.copy(test_image_path, onnx_test_image_path)
if not osp.exists(onnx_test_image_path):
    raise FileNotFoundError(f"{onnx_test_image_path} was not found")
    
# inicialize onnx model
onnx_file = os.path.join(tmp_entities.classifier_onnx_files, out_onnx_filename)
if not osp.exists(onnx_file):
    raise FileNotFoundError(f"{onnx_file} was not found")
    
ort_session = onnxruntime.InferenceSession(onnx_file, providers=['CPUExecutionProvider'])
input_name = ort_session.get_inputs()[0].name
output_name = [out.name for out in ort_session.get_outputs()]

# read test image and processing for inference by onnx
pre_post_processing = PrePostProcessing()
input_data = pre_post_processing.prep_processing(onnx_test_image_path, input_size=INPUT_SIZE)
# inference onnx by test image
outs = ort_session.run(output_name, {input_name: input_data})
outs = pre_post_processing.post_processing(outs, categories=CATEGORIES)

# save and reopen pickle file output of inference by test image
with open(osp.join(tmp_entities.classifier_onnx_files, 'test_result.pkl'), 'wb') as pkl_file:
    pickle.dump(outs, pkl_file)    
with open(osp.join(tmp_entities.classifier_onnx_files, 'test_result.pkl'), 'rb') as f_id:
    test_result = f_id.read()    
    
# open test image
with open(onnx_test_image_path, 'rb') as f_id:
    test_image = f_id.read()   

#### Packaging obj_detector files to bento_service artifacts

In [ ]:
model_service = ModelService()
model_service.pack('model', onnx_file)
model_service.pack('test_image', test_image)
model_service.pack('test_result', test_result)    
model_service.pack('categories', CATEGORIES)
model_service.pack('input_size', INPUT_SIZE)

In [ ]:
model_service.predict(onnx_test_image_path)

### Send packaged onnx_obj_detector to outputs

In [ ]:
# save model as a bento pack
from sinara.bentoml import save_bentoservice
save_bentoservice(model_service, path=outputs.bento_service, substep=substep)

In [ ]:
# stop spark
SinaraSpark.stop_session()